In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import PillowWriter
from mpl_toolkits.mplot3d import Axes3D
import scienceplots
from scipy.sparse.linalg import eigsh   #Scipy稀疏线性代数库
from scipy.sparse.linalg import eigs    #Scipy稀疏线性代数库
plt.style.use(['science', 'notebook'])
from scipy import sparse
from skimage import measure
import torch
import plotly.graph_objects as go
from torch import lobpcg
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

因为势能表达为$$V(r) = -\frac{e^2}{4 \pi \epsilon_0 r} = -\frac{1}{m a_0 r}$$
以及三维薛定谔方程的形式为：$$\left[-\frac{1}{2}(D \oplus D \oplus D) + m(\Delta x)^2 V \right] \psi = \left(m (\Delta x)^2 E\right) \psi$$
所以可以使用玻尔半径$a_0$化为无量纲版本：
$$m (\Delta x)^2 V = -\frac{(\Delta x)^2}{a_0 r} = -\frac{(\Delta x/a_0)^2}{(r/a_0)}$$

In [4]:
# 初始化一个3D网格
N=120
X, Y, Z = np.mgrid[-25:25:N*1j, -25:25:N*1j, -25:25:N*1j]
dx=np.diff(X[:,0,0])[0]

In [5]:
# 设置势能
def get_potential(x,y,z):
    return - dx**2 / np.sqrt(x**2 + y**2 + z**2 + 1e-10)
V=get_potential(X,Y,Z)
V.shape

(120, 120, 120)

动能项为：
$$T=-\frac{1}{2}(D\otimes D \otimes D)$$
势能项为：
$$U=m(\Delta x)^2V$$
哈密顿量为：
$$H=T+U$$

In [8]:
# 定义哈密顿量
diag=np.ones(N)
diags=np.array([diag,-2*diag,diag])
D=sparse.spdiags(diags,np.array([-1,0,1]),N,N)
T=-1/2 * sparse.kronsum(sparse.kronsum(D,D),D)
U=sparse.diags(V.reshape(N**3),0)
H=T+U
H.shape

(1728000, 1728000)

In [9]:
# 由scipy的稀疏矩阵转换为torch的稀疏张量
H=H.tocoo()
H=torch.sparse_coo_tensor(indices=torch.tensor([H.row,H.col]),values=torch.tensor(H.data),size=H.shape).to(device)

C:\Users\xsxwz\AppData\Local\Temp\ipykernel_13620\1749318383.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:278.)
  H=torch.sparse_coo_tensor(indices=torch.tensor([H.row,H.col]),values=torch.tensor(H.data),size=H.shape).to(device)


In [11]:
# 通过Locally Optimal Block Preconditioned Conjugate Gradient Method方法计算特征值和特征向量
eigenvalues, eigenvectors = lobpcg(H, k=20, largest=False)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
